# Chapter 3 - When Objects are alike

Duplicated code is considered evil. Steps to avoid duplicate code should be taken wherever possible.

There are many ways to merge code or objects that function similarly.

This chapter will focus on inheritance.

Specifically we will cover:

* Basic inheritance
* Inheriting from built-ins
* Multiple inheritance
* Polymorphism and duck typing

## Basic Inheritance

Technically every class we create uses inheritance.

All python classes are subclasses of the special class named _object_.

By default all classes inherit the object class.



In [1]:
class MySubClass(object):
    pass

# IS exactly the same as

class MySubClass():
    pass

A class being ingerited from is called a superclass or parent class.

In the previous example the superclass is _object_. Even when not explicitly called.

To invoke inheritance simply include the name of the class in the paranthesis of the class definition.

In practice inheritance is used to provide additional functionality to an existing class.

**Example**

A contact manager that tracks the names and emails of people

In [1]:
class Contact:
    all_contacts = []
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

The _all_contacts_ list is shared by all instances of this class. Because it is part of class definition.

The following can be used to access the all_contacts object

```python
Contact.all_contacts

# alternatively

self.all_contacts # will work on any object instantiated from Contact
```

If a field is not found on the object then python will look in the class definition for it.

> Warning
>
> Using self.all_contacts in the class definition will create a **new** instance variable. This will then only be acessible on that particular object.

Lets make a supplier class that inherits from the Contact class. We would like to add an order method, however it doesn't make sense to be able to order from everyone in the address book, it would only apply to suppliers.